In [7]:
import re
re.sub(r'(?:,|\+)','','4343,433+')

res=re.findall(r"'(\w+)'","sddf ;'url'sad")
res[0]

bool(re.search(r'\d','dd1 f'))


True

In [2]:
# 装饰器
"""
装饰器本质上是一个 Python 函数或类，它可以让其他函数或类在不需要做任何代码修改的
前提下增加额外功能，装饰器的返回值也是一个函数/类对象。它经常用于有切面需求的场景，
比如：插入日志、性能测试、事务处理、缓存、权限校验等场景，装饰器是解决这类问题的
绝佳设计。有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码到装饰器中
并继续重用。概括的讲，装饰器的作用就是为已经存在的对象添加额外的功能。
"""
#装饰器不带参数
from functools import wraps
def log(func):
    
    # wraps需要把原始函数的__name__等属性复制到wrapper()函数中，
    # 否则，有些依赖函数签名的代码执行就会出错。
    @wraps(func)
    def wrapper(*args, **kargs):
        print(args,kargs)
        return func(*args, **kargs)
    return wrapper


@log
def now(name,is_super):
    print(name)

now('xander',is_super=False)

('xander',) {'is_super': False}
xander


In [3]:
#装饰器带参数
def log(*a,**ka):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kargs):
            print(a,ka)
            print(args,kargs)
            return func(*args, **kargs)
        return wrapper
    return decorator

@log('od',action='execute')
def now(name,is_super):
    print(name)

now('xander',is_super=False)

('od',) {'action': 'execute'}
('xander',) {'is_super': False}
xander


In [183]:
import json

#装饰器带参数
def cache(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kargs):
            args_str = ''.join([str(i) for i in args])
            kargs_str = ",".join([f"{k}={v}" for k, v in kargs.items()])
            key = f'{func.__name__}_{args_str}_{kargs_str}'
            res = REDIS_CLIENT.get(key)
            if res:
                return json.loads(res)
            res = func(*args, **kargs)
            data = json.dumps(res)
            REDIS_CLIENT.set(key, data, seconds)
            return res
        return wrapper
    return decorator

class Test():
    @cache(5)
    def test(self, x):
        return str(datetime.datetime.now()), True

t = Test()

In [87]:
#class 装饰器
class logit():
    
    def __call__(self,func):
    
        @wraps(func)
        def wrapper(*args,**kargs):
            print(args,kargs)
            return func(*args,**kargs)
        return wrapper
    
@logit()    
def test(status):
    pass
    
test('success')

('success',) {}


In [27]:
# class 装饰器传参数
from functools import wraps

class Logit():
    def __init__(self,*a,**ka):
        self.a=a
        self.ka=ka
    
    def __call__(self,func):
        @wraps(func)
        def wrapper(*args,**kargs):
            print('logit params',self.a,self.ka)
            print('func params',args,kargs)
            return func(*args,**kargs)
        return wrapper

@Logit('abc',is_super=True)  
def test(*args,**kargs):
    print('test',args,kargs)

test('efg',path='a/b/c')

logit params ('abc',) {'is_super': True}
func params ('efg',) {'path': 'a/b/c'}
test ('efg',) {'path': 'a/b/c'}


In [ ]:
class Test():
    cache_data = {}

    def cache_property(func):
        @wraps(func)
        def wrapper(*args, **kargs):
            self = args[0]
            if func.__name__ not in self.cache_data:
                self.cache_data[func.__name__] = func(*args, **kargs)
            return self.cache_data[func.__name__]

        return wrapper

    @property
    @cache_property
    def info(self):
        print('info func')
        return 'a'

In [89]:
f=filter(lambda x:x.startswith('abc'),['abcdsf','did'])
list(f)

['abcdsf']

In [28]:
a={1,2,4,5}
b={4,5,6}
a.symmetric_difference(b)

{1, 2, 6}

In [30]:
exec('print(1+2)')

3


In [43]:
# 线程安全的单例
import threading

def lock_new(func):
    func.lock = threading.Lock()
    def wrapper(*args,**kargs):
        with func.lock:
            return func(*args,**kargs)
        return wrapper
    
class SingleObject():
    obj = None
    def __new__(cls,*args,**kargs):
        if cls.obj is None:
            cls.obj = super(SingleObject,cls).__new__(cls,*args,**kargs)
        return cls.obj

SingleObject(),SingleObject()

(<__main__.SingleObject at 0x107fe2ba8>,
 <__main__.SingleObject at 0x107fe2ba8>)

In [47]:
a=[1,3]
b=[4,56]
c=[7,8,9]
a.extend(b)
b.append(c)
a,b

([1, 3, 4, 56], [4, 56, [7, 8, 9]])

In [56]:
#切片操作： a[stat:end:step]
a=[1,3,7,5,0,8]
a[::]#[1, 3, 7, 5, 0, 8]
a[::-1]#[8, 0, 5, 7, 3, 1]
a[::2]#[1, 7, 0]
a[len(a)::]=[10]#[1, 3, 7, 5, 0, 8, 10]
a[2:2]=[99]#[1, 3, 99, 7, 5, 0, 8, 10]
a[::2]=[0]*4#[0, 3, 0, 7, 0, 0, 0, 10]
a[:3]=[]#[7, 0, 0, 0, 10]
del a[:3]#[0, 10]
a

[0, 10]

In [11]:
import re
re_compile=re.compile(r'^(?![0-9]+$)(?![a-zA-Z]+$)[0-9A-Za-z!@#$%^&*()-=_+|\\\{\}\[\];\':\",\.<>\?]{8,20}$',re.VERBOSE )
# re_compile=re.compile(r'(^.{8,20}$)(?=[a-z]+).*$',re.VERBOSE)
# re_compile=re.compile(r'(?=^.{8,20}$)((?=.*\d+))(?![.\n])(?=.*[A-Z])(?=.*[a-z]).*$')
re_compile.match('qwerasd3')

<_sre.SRE_Match object; span=(0, 8), match='qwerasd3'>

In [12]:
#正则
def check_password(password):
#     passwordReg = re.compile(r'(?=^.{8,20}$)((?=.*\d+))(?=.*[a-zA-Z]).*$', re.VERBOSE)
#     passwordReg = re.compile(r'(?=^.{8,20}$)((?=.*\d+))(?=.*[\!@#\$%^&\*()_\-\+\=`~|\[\]\{\};:,.\?/]).*$', re.VERBOSE)
#     passwordReg = re.compile(r'(?=^.{8,20}$)(?=.*[a-zA-Z])(?=.*[\!@#\$%^&\*()_\-\+\=`~|\[\]\{\};:,.\?/]).*$', re.VERBOSE)
    re_compile = re.compile(
    r"""
    (?=^.{8,20}$)(
        ((?=.*\d+))(?=.*[a-zA-Z]) |
        ((?=.*\d+))(?=.*[\!@#\$%^&\*()_\-\+\=`~|\[\]\{\};:,.\?/]) |
        (?=.*[a-zA-Z])(?=.*[\!@#\$%^&\*()_\-\+\=`~|\[\]\{\};:,.\?/])
    ).*$
    """, re.VERBOSE
)
    match = re_compile.match(password)
    return match is not None
check_password("234#1234")

True

In [6]:
#整理歌曲文件夹
import os
path = '/Users/yetongxue/Downloads'
sub_dirs = [(x[0],x[2]) for x in os.walk(path, topdown=False)]
for file in sub_dirs[1:]:
    base_dir = file[0]
    for f in file[1]:
        origin = os.path.join(base_dir, f)
        new = os.path.join(path, f)
        os.rename(origin, new)

In [10]:
from functools import wraps

def log(func):
    
    @wraps(func)
    def wrapper(*args, **kargs):
        print('action')
        return func(*args, **kargs)
    return wrapper


@log
def test(a):
    print(a)

test('1')

def log(*a, **ka):
    def deractor(func):
        print(a, ka)
        
        @wraps(func)
        def wrapper(*args, **kargs):
            return func(*args, **kargs)
        return wrapper
    return deractor

@log('log_p')
def test2(a):
    print(a)
    
test2('ok')

action
1
('log_p',) {}
ok
